In [ ]:
# Variables

# MongoDB database location in this file
PATH = '/scratch/ob2205/personal/ip/ip_address.txt'

In [1]:
!pip install sparknlp pymongo

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.jars.packages', 
         "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,com.databricks:spark-xml_2.12:0.18.0,com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3")
conf.set('spark.driver.memory','8g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)
spark

:: loading settings :: url = jar:file:/ext3/spark-3.3.0-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ob2205/.ivy2/cache
The jars for the packages stored in: /home/ob2205/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-06426938-450b-4754-bc4f-d049066bac59;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.type

24/04/21 09:57:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/ext3/pyspark/lib/python3.10/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [41]:
ip_df = spark.read.text(PATH)
ip_address = ip_df.head(1)[0].value
ip_address

'10.32.32.5:27040'

In [42]:
spark.sparkSession

In [43]:
from pymongo import MongoClient

client = MongoClient(host=[f"{ip_address}"])

In [44]:
client.server_info()

{'version': '7.0.4',
 'gitVersion': '38f3e37057a43d2e9f41a39142681a76062d582e',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [7, 0, 4, 0],
 'openssl': {'running': 'OpenSSL 3.0.2 15 Mar 2022',
  'compiled': 'OpenSSL 3.0.2 15 Mar 2022'},
 'buildEnvironment': {'distmod': 'ubuntu2204',
  'distarch': 'x86_64',
  'cc': '/opt/mongodbtoolchain/v4/bin/gcc: gcc (GCC) 11.3.0',
  'ccflags': '-Werror -include mongo/platform/basic.h -ffp-contract=off -fasynchronous-unwind-tables -g2 -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -gdwarf-5 -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=sandybridge -mtune=generic -mprefer-vector-width=128 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -gdwarf64 -Wa,--nocompress-debug-sections -fno-builtin-memcmp -Wimplicit-fallthrough=5',
  'cxx': '/opt

In [ ]:
db=client.test
db

In [ ]:
# Get Scala Version
sc._gateway.jvm.scala.util.Properties.versionString()

In [ ]:
import os
from zipfile import ZipFile
source_path = "/scratch/work/public/proquest/proquest_hnp/BostonGlobe/BG_20151210212722_00001.zip"
with ZipFile(source_path, "r") as zip:
    print(zip.infolist())
    zip.extract('250949924.xml', 'zip_tmp')

In [ ]:
df = spark.read\
    .option('rootTag', 'Record')\
    .option('rowTag', 'Record')\
    .format("xml").load("zip_tmp/250949924.xml")

In [ ]:
df.printSchema()

In [ ]:
df.select('RecordID', 'Publisher', 'AlphaPubDate', 'NumericPubDate', 'RecordTitle', 'SourceType').show(2)

In [ ]:
# Save to mongo
df.write.format('com.mongodb.spark.sql.DefaultSource')\
    .option('uri',f'mongodb://{ip_address}/test.newspapers')\
    .save()

In [ ]:
db.list_collection_names()

In [ ]:
db.newspapers.find_one()

In [ ]:
import sparknlp

from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *

spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

In [ ]:
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)